In [3]:
import pyaudio
import numpy as np
import librosa, librosa.display
import IPython
import tensorflow as tf
import warnings
import statistics as st
warnings.filterwarnings('ignore')

In [4]:
FORMAT = pyaudio.paFloat32
CHANNELS = 1
RATE = 48000
CHUNK = int(RATE * 0.04)

buffer = np.empty(25)
buffer_index = 0

model = tf.keras.models.load_model("../saved_models/4_classes-01_24_2023_14_05_19")

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

while (True):
    data = np.frombuffer(stream.read(CHUNK), 'float32')

    data = (data - min(data)) / (np.max(data) - np.min(data))

    mfcc = np.array(librosa.feature.mfcc(y=data, sr=RATE, n_mels=128, n_mfcc=12, n_fft=512, hop_length=128))
    mfcc_mean = mfcc.mean(axis=1)
    mfcc_min = mfcc.min(axis=1)
    mfcc_max = mfcc.max(axis=1)
    mfcc_feature = np.concatenate( (mfcc_mean, mfcc_min, mfcc_max) )

    x = model.predict(mfcc_feature.reshape(1, len(mfcc_feature)), verbose=False)
    
    IPython.display.clear_output(wait=True)
    
    prediction = np.argmax(x)

    buffer[buffer_index] = prediction
    buffer_index += 1
    if buffer_index == 25:
        buffer_index = 0

    running_prediction = st.mode(buffer)
    print(buffer, running_prediction)

    if running_prediction == 0:
        print("Adam")
    elif running_prediction == 1:
        print("Aiden")
    elif running_prediction == 2:
        print("Hannah")
    elif running_prediction == 3:
        print("Parker")

[2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 1. 2. 1. 3. 2. 2.
 2.] 2.0
Hannah


KeyboardInterrupt: 